<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Estudiante:

- Mario González Otero

## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** [ENLACE](https://github.com/mgzotero/MDS7202)

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
import optuna as opt
import pickle
import warnings
import copy

In [2]:
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("sales.csv")
df["date"] = pd.to_datetime(df["date"])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [4]:
SEED = 42

In [5]:
def get_train_val_test(
    df: pd.DataFrame, target_col: str, data_size: tuple[float], seed: int
) -> tuple[pd.DataFrame]:
    """Divide el conjunto de datos en entrenamiento, validación y prueba.

    Args:
        df (pd.DataFrame): Conjunto de datos.
        target_col (str): Nombre de la columna objetivo.
        data_size (tuple[float]): Tamaño de los conjuntos de datos de validación y prueba.
        seed (int): Semilla aleatoria.

    Returns:
        tuple[pd.DataFrame]: Conjuntos de datos de entrenamiento, validación y prueba.
    """

    # Copia del dataframe
    df = df.copy(deep=True)

    # Tamaños reales de los conjuntos de datos
    _, val_size, test_size = data_size
    val_test_size = val_size + test_size
    test_size = test_size / val_test_size

    # Primer split
    X_train, X_val_test, y_train, y_val_test = train_test_split(
        df.drop(target_col, axis=1),
        df[target_col],
        test_size=val_test_size,
        random_state=seed,
    )

    # Segundo split
    X_val, X_test, y_val, y_test = train_test_split(
        X_val_test,
        y_val_test,
        test_size=test_size,
        random_state=seed,
    )

    return X_train, X_val, X_test, y_train, y_val, y_test

In [6]:
X_train, X_val, X_test, y_train, y_val, y_test = get_train_val_test(
    df=df, target_col="quantity", data_size=(0.7, 0.2, 0.1), seed=SEED
)

print(
    f"Train size: {len(X_train)/len(df):.2f}",
    f"Validation size: {len(X_val)/len(df):.2f}",
    f"Test size: {len(X_test)/len(df):.2f}",
    sep="\n",
)

Train size: 0.70
Validation size: 0.20
Test size: 0.10


In [7]:
def get_day_month_year(
    df: pd.DataFrame,
    date_col: str,
    year_col: str = "year",
    month_col: str = "month",
    day_col: str = "day",
) -> pd.DataFrame:
    """Extrae el día, mes y año de una columna de fecha.

    Args:
        df (pd.DataFrame): Conjunto de datos.
        date_col (str): Nombre de la columna de fecha.
        day_col (str, opcional): Nombre de la columna de día. Por defecto es "day".
        month_col (str, opcional): Nombre de la columna de mes. Por defecto es "month".
        year_col (str, opcional): Nombre de la columna de año. Por defecto es "year".

    Returns:
        pd.DataFrame: Conjunto de datos con las columnas de día, mes y año.
    """

    df = df.copy(deep=True)
    df[[year_col, month_col, day_col]] = (
        df[date_col]
        .astype(str)
        .str.split("-", expand=True)
        .astype(int)
    )

    return df[[year_col, month_col, day_col]]

In [8]:
# Columnas
date_cols = ["date"]
cat_cols = ["city", "shop", "brand", "container", "capacity"]
num_cols = ["lat", "long", "pop", "price"]
cols = date_cols + cat_cols + num_cols

# Definimos el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ("ft", FunctionTransformer(get_day_month_year, kw_args={"date_col": "date"}, validate=False), date_cols),
        ("onehot", OneHotEncoder(sparse=False, handle_unknown="ignore"), cat_cols),
        ("minmax", MinMaxScaler(), num_cols),
    ]
).set_output(transform="pandas")

# Entrenamos el preprocesador
preprocessor.fit_transform(X_train).head()

,ft__year,ft__month,ft__day,onehot__city_Athens,onehot__city_Irakleion,onehot__city_Larisa,onehot__city_Patra,onehot__city_Thessaloniki,onehot__shop_shop_1,onehot__shop_shop_2,...,onehot__container_can,onehot__container_glass,onehot__container_plastic,onehot__capacity_1.5lt,onehot__capacity_330ml,onehot__capacity_500ml,minmax__lat,minmax__long,minmax__pop,minmax__price
292,2012,4,30,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.548667,0.000000,0.055829,0.519231
3366,2015,2,28,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.498817,0.581343,0.990904,0.128205
3685,2015,6,30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.495619,0.572795,0.990904,0.117521
2404,2014,4,30,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.498817,0.581343,0.992700,0.040598
2855,2014,9,30,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.000000,0.003686,0.096154


In [9]:
new_cols = preprocessor.transform(X_train).columns
new_cols

Index(['ft__year', 'ft__month', 'ft__day', 'onehot__city_Athens',
       'onehot__city_Irakleion', 'onehot__city_Larisa', 'onehot__city_Patra',
       'onehot__city_Thessaloniki', 'onehot__shop_shop_1',
       'onehot__shop_shop_2', 'onehot__shop_shop_3', 'onehot__shop_shop_4',
       'onehot__shop_shop_5', 'onehot__shop_shop_6',
       'onehot__brand_adult-cola', 'onehot__brand_gazoza',
       'onehot__brand_kinder-cola', 'onehot__brand_lemon-boost',
       'onehot__brand_orange-power', 'onehot__container_can',
       'onehot__container_glass', 'onehot__container_plastic',
       'onehot__capacity_1.5lt', 'onehot__capacity_330ml',
       'onehot__capacity_500ml', 'minmax__lat', 'minmax__long', 'minmax__pop',
       'minmax__price'],
      dtype='object')

### DummyRegressor

In [10]:
# Definición del modelo
dummyreg = DummyRegressor(strategy="mean")

# Pipeline
dummypipe = make_pipeline(preprocessor, dummyreg)

# Entrenamiento del modelo
dummypipe.fit(X_train, y_train)

# Mean absolute error
mae = mean_absolute_error(y_val, dummypipe.predict(X_val))
print(f"MAE: {mae:.2f}")

MAE: 13298.50


### XGBRegressor

In [11]:
# Definimos el modelo
xgbreg1 = xgb.XGBRegressor()

# Pipeline
xgbpipe1 = make_pipeline(preprocessor, xgbreg1)

# Entrenamos el pipeline
xgbpipe1.fit(X_train, y_train)

# Mean absolute error
mae = mean_absolute_error(y_val, xgbpipe1.predict(X_val))
print(f"MAE: {mae:.2f}")

MAE: 2440.84


Como acabamos de observar, el modelo `XGBRegressor` es mejor que el `DummyRegressor` en términos de MAE. La interpretación de este error es que, en promedio, el modelo `Dummy` se equivoca en aproximadamente 13300 unidades al predecir la demanda de la empresa. Por otro lado, el modelo `XGB` se equivoca en aproximadamente 2400 unidades.

In [12]:
# Guardamos los modelos
with open("dummyreg.pkl", "wb") as f:
    pickle.dump(dummypipe, f)
with open("xgbreg1.pkl", "wb") as f:
    pickle.dump(xgbpipe1, f)

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [13]:
# Instanciamos el XGBRegressor con restricciones
xgbreg2 = xgb.XGBRegressor(monotone_constraints={"minmax__price": -1})

# Definimos el pipeline
xgbpipe2 = make_pipeline(preprocessor, xgbreg2)

# Entrenamos el pipeline
xgbpipe2.fit(X_train, y_train)

# Calculamos el MAE
mae = mean_absolute_error(y_val, xgbpipe2.predict(X_val))
print(f"MAE: {mae:.2f}")

MAE: 2446.17


In [14]:
# Guardamos el modelo
with open("xgbreg2.pkl", "wb") as f:
    pickle.dump(xgbpipe2, f)

Si bien el error de validación empeora al incluir esta restricción, esta sí hace sentido teórico, por lo que es posible que la restricción sea poco compatible con la combinación de hiperparámetros por defecto o, simplemente, que el modelo no sea capaz de incluir esta restricción de buena manera debido a la presencia de ciertos productos con elasticidades un tanto erráticas.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [15]:
def objective_function(trial):
    preprocessor = ColumnTransformer(
        transformers=[
            ("ft", FunctionTransformer(get_day_month_year, kw_args={"date_col": "date"}, validate=False), date_cols),
            ("onehot", OneHotEncoder(sparse=False, handle_unknown="ignore"), cat_cols),
            ("minmax", MinMaxScaler(), num_cols),
        ]
    ).set_output(transform="pandas")

    params = {
        "xgbregressor__objective": "reg:squarederror",
        "xgbregressor__eval_metric": "mae",
        "xgbregressor__learning_rate": trial.suggest_float("xgbregressor__learning_rate", 0.001, 0.1),
        "xgbregressor__n_estimators": trial.suggest_int("xgbregressor__n_estimators", 50, 1000),
        "xgbregressor__max_depth": trial.suggest_int("xgbregressor__max_depth", 3, 10),
        "xgbregressor__max_leaves": trial.suggest_int("xgbregressor__max_leaves", 0, 100),
        "xgbregressor__min_child_weight": trial.suggest_int("xgbregressor__min_child_weight", 1, 5),
        "xgbregressor__alpha": trial.suggest_float("xgbregressor__alpha", 0, 1),
        "xgbregressor__lambda": trial.suggest_float("xgbregressor__lambda", 0, 1),
        "columntransformer__onehot__min_frequency": trial.suggest_float("columntransformer__onehot__min_frequency", 0, 1),
    }

    xgbpipe = make_pipeline(preprocessor, xgb.XGBRegressor(seed=SEED)).set_params(**params)

    xgbpipe.fit(X_train, y_train)
    metric = mean_absolute_error(y_val, xgbpipe.predict(X_val))

    return metric


study = opt.create_study(direction="minimize", sampler=opt.samplers.TPESampler(seed=SEED))
study.optimize(objective_function, n_trials=500, timeout=300)

[I 2023-11-18 23:05:03,661] A new study created in memory with name: no-name-fb13a31c-b03f-47a2-8eca-5c07e229f3bb
[I 2023-11-18 23:05:08,515] Trial 0 finished with value: 7294.470116106797 and parameters: {'xgbregressor__learning_rate': 0.03807947176588889, 'xgbregressor__n_estimators': 954, 'xgbregressor__max_depth': 8, 'xgbregressor__max_leaves': 60, 'xgbregressor__min_child_weight': 1, 'xgbregressor__alpha': 0.15599452033620265, 'xgbregressor__lambda': 0.05808361216819946, 'columntransformer__onehot__min_frequency': 0.8661761457749352}. Best is trial 0 with value: 7294.470116106797.
[I 2023-11-18 23:05:10,300] Trial 1 finished with value: 2870.9955459482153 and parameters: {'xgbregressor__learning_rate': 0.06051038616257767, 'xgbregressor__n_estimators': 723, 'xgbregressor__max_depth': 3, 'xgbregressor__max_leaves': 97, 'xgbregressor__min_child_weight': 5, 'xgbregressor__alpha': 0.21233911067827616, 'xgbregressor__lambda': 0.18182496720710062, 'columntransformer__onehot__min_frequen

In [16]:
study.best_params

{'xgbregressor__learning_rate': 0.06905349647134347,
 'xgbregressor__n_estimators': 745,
 'xgbregressor__max_depth': 7,
 'xgbregressor__max_leaves': 75,
 'xgbregressor__min_child_weight': 5,
 'xgbregressor__alpha': 0.8888949069410164,
 'xgbregressor__lambda': 0.2806339570986051,
 'columntransformer__onehot__min_frequency': 0.09807849584586154}

In [17]:
preprocessor_opt1 = copy.deepcopy(preprocessor)

xgbpipe3 = make_pipeline(preprocessor_opt1, xgb.XGBRegressor(seed=SEED)).set_params(
    **study.best_params
)
xgbpipe3.fit(X_train, y_train)
mae = mean_absolute_error(y_val, xgbpipe3.predict(X_val))
print(f"MAE: {mae:.2f}")

MAE: 2024.06


In [18]:
print(f"Número de trials: {len(study.trials)}")

Número de trials: 80


In [19]:
# Guardamos el modelo
with open("xgbreg3.pkl", "wb") as f:
    pickle.dump(xgbpipe3, f)

Comparando el modelo con la versión con hiperparámetros por defecto, esta versión ocupando optimización de hiperparámetros logra mejorar bastante el `MAE`, bajando aproximadamente 400 unidades. Esto se debe a que, al optimizar los hiperparámetros, se logra encontrar una mejor combinación de estos, lo que permite que el modelo se ajuste mejor a los datos.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

El prunning en contexto de optimización de hiperparámetros se refiere a la técnica de detener el entrenamiento de un modelo cuando se detecta que este no está mejorando. En este caso, el prunning se implementa en el método `optuna.integration.XGBoostPruningCallback` y se utiliza en conjunto con el método `optimize` de `optuna`. En este caso, el prunning debería impactar en el entrenamiento de manera positiva, ya que permite detener el entrenamiento de un modelo cuando este no está mejorando, lo que permite ahorrar tiempo de entrenamiento y, por ende, recursos computacionales.

In [20]:
opt.logging.set_verbosity(opt.logging.WARNING)

In [ ]:
def objective_function(trial):
    preprocessor = ColumnTransformer(
        transformers=[
            ("ft", FunctionTransformer(get_day_month_year, kw_args={"date_col": "date"}, validate=False), date_cols),
            ("onehot", OneHotEncoder(sparse=False, handle_unknown="ignore"), cat_cols),
            ("minmax", MinMaxScaler(), num_cols),
        ]
    ).set_output(transform="pandas")

    params = {
        "xgbregressor__objective": "reg:squarederror",
        "xgbregressor__eval_metric": "mae",
        "xgbregressor__learning_rate": trial.suggest_float("xgbregressor__learning_rate", 0.001, 0.1),
        "xgbregressor__n_estimators": trial.suggest_int("xgbregressor__n_estimators", 50, 1000),
        "xgbregressor__max_depth": trial.suggest_int("xgbregressor__max_depth", 3, 10),
        "xgbregressor__max_leaves": trial.suggest_int("xgbregressor__max_leaves", 0, 100),
        "xgbregressor__min_child_weight": trial.suggest_int("xgbregressor__min_child_weight", 1, 5),
        "xgbregressor__alpha": trial.suggest_float("xgbregressor__alpha", 0, 1),
        "xgbregressor__lambda": trial.suggest_float("xgbregressor__lambda", 0, 1),
        "columntransformer__onehot__min_frequency": trial.suggest_float("columntransformer__onehot__min_frequency", 0, 1),
    }

    pruning_callback = opt.integration.XGBoostPruningCallback(trial, "validation_0-mae")

    xgbpipe = make_pipeline(preprocessor, xgb.XGBRegressor(seed=SEED)).set_params(**params)
    
    preprocessor.fit(X_train)
    xgbpipe.fit(X_train, y_train, xgbregressor__callbacks=[pruning_callback], xgbregressor__eval_set=[(preprocessor.transform(X_val), y_val)])
    metric = mean_absolute_error(y_val, xgbpipe.predict(X_val))

    return metric

study = opt.create_study(direction="minimize", sampler=opt.samplers.TPESampler(seed=SEED))
study.optimize(objective_function, n_trials=500, timeout=300, show_progress_bar=True)

In [22]:
study.best_params

{'xgbregressor__learning_rate': 0.09703848850762908,
 'xgbregressor__n_estimators': 433,
 'xgbregressor__max_depth': 7,
 'xgbregressor__max_leaves': 34,
 'xgbregressor__min_child_weight': 4,
 'xgbregressor__alpha': 0.6052525011307642,
 'xgbregressor__lambda': 0.2350573683327373,
 'columntransformer__onehot__min_frequency': 0.06657417026187086}

In [23]:
preprocessor_opt2 = copy.deepcopy(preprocessor)

xgbpipe4 = make_pipeline(preprocessor_opt2, xgb.XGBRegressor(seed=SEED)).set_params(
    **study.best_params
)

xgbpipe4.fit(X_train, y_train)
mae = mean_absolute_error(y_val, xgbpipe4.predict(X_val))
print(f"MAE: {mae:.2f}")

MAE: 2077.64


In [24]:
print(f"Número de trials: {len(study.trials)}")

Número de trials: 172


In [25]:
# Guardamos el modelo
with open("xgbreg4.pkl", "wb") as f:
    pickle.dump(xgbpipe4, f)

Evidentemente, los resultados en cuanto a `MAE` empeoran un poco, pero esto se debe a que el prunning detiene el entrenamiento de un modelo cuando este no está mejorando, por lo que el modelo no alcanza a entrenar por el tiempo especificado en el método `optimize` de `optuna`. Sin embargo, si dejáramos que la optimización con `prunning` se ejecutara por más tiempo, es probable que los resultados mejoren.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [26]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [27]:
# Gráfico de historial de optimización
plot_optimization_history(study)

Del gráfico anterior, es posible observar que ya a partir del primer trial se observa una bajada importante en el valor de la función objetivo, aunque los mejores resultados se obtienen a partir del trial 13, en donde la pérdida bordea un `MAE` de 2000.

In [28]:
# Gráfico de coordenadas paralelas
plot_parallel_coordinate(study)

Del gráfico anterior es posible notar que la mayoría de los trials se concentran en valores de `min_frequency` cercanos a 0.1, `learning_rate` cercanos a 0.1, `max_depth` cercanos a 7, `max_leaves` cercanos a 30, `reg_alpha` cercanos a 0.6 y `reg_lambda` cercanos a 0.2.

In [29]:
# Gráfico de importancia de parámetros
plot_param_importances(study)

Del gráfico anterior es posible observar que los hiperparámetros `min_frequency`, `learning_rate` y `n_estimators` son los que más impactan en la optimización del modelo.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [30]:
mae1 = mean_absolute_error(y_val, dummypipe.predict(X_val))
mae2 = mean_absolute_error(y_val, xgbpipe1.predict(X_val))  # XBRegressor por defecto
mae3 = mean_absolute_error(y_val, xgbpipe2.predict(X_val))  # XBRegressor con restricciones
mae4 = mean_absolute_error(y_val, xgbpipe3.predict(X_val))  # XBRegressor con optimización
mae5 = mean_absolute_error(y_val, xgbpipe4.predict(X_val))  # XBRegressor con optimización y prunning

mae_df = pd.DataFrame(
    {
        "Modelo": ["DummyRegressor", "XGBRegressor (por defecto)", "XGBRegressor (con restricciones)", "XGBRegressor (con optimización)", "XGBRegressor (con optimization y prunning)"],
        "MAE": [mae1, mae2, mae3, mae4, mae5],
    }
)

mae_df

,Modelo,MAE
0,DummyRegressor,13298.497767
1,XGBRegressor (por defecto),2440.837208
2,XGBRegressor (con restricciones),2446.169878
3,XGBRegressor (con optimización),2024.061063
4,XGBRegressor (con optimization y prunning),2077.635090


Evidentemente, el mejor modelo es el que se obtiene al implementar `XGBRegressor` con `optuna` pero sin `prunning`, ya que este obtiene el menor `MAE` de todos los modelos entrenados. Sin embargo, es posible notar que el modelo que se obtiene al implementar `XGBRegressor` con `optuna` y `prunning` obtiene un `MAE` muy similar al del modelo anterior, por lo que es posible que este último modelo sea más eficiente en términos de recursos computacionales.

In [31]:
mae_test = mean_absolute_error(y_test, xgbpipe4.predict(X_test))
print(f"MAE: {mae_test:.2f}")

MAE: 1956.41


Es posible notar que en el conjunto de `test` el modelo obtiene mejores resultados en cuanto a `MAE`, lo que es normal, pues los conjuntos de `test` y `validación` no son exactamente iguales debido al ruido propio del muestreo. Sin embargo, es posible notar que el `MAE` obtenido en el conjunto de `test` es muy similar al `MAE` obtenido en el conjunto de `validación`.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>